In [11]:
!pip3 install tensorflow

  Could not find a version that satisfies the requirement tensorflow (from versions: )
No matching distribution found for tensorflow


In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from datetime import datetime
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

SyntaxError: invalid syntax (pywrap_tensorflow_internal.py, line 114)

In [49]:
#Daten one hot kodieren für Modell 1
df = pd.read_csv("Data/daten_anonym2/arzta_daten_anonym1.csv", sep=';').iloc[:100, :]

## transfrom str type to float type
columns_comma = ['RECHNUNGSBETRAG', 'FAKTOR', 'BETRAG', 'ALTER', 'KORREKTUR'] 
df[columns_comma] = df[columns_comma].apply(lambda x: x.str.replace(',', '.'))
for column in columns_comma:
    df[column] = pd.to_numeric(df[column], downcast='float')
    
    
# df1 = pd.get_dummies(df['NUMMER_KAT'])
# df = df.drop(['NUMMER_KAT'], axis=1)
# data = df.merge(df1, left_index=True, right_index=True)
# df2 = pd.get_dummies(df['NUMMER'])
# df = df.drop(['NUMMER'], axis=1)
# data = data.merge(df2, left_index=True, right_index=True)
# df3 = pd.get_dummies(df['LEISTUNG'])
# df = df.drop(['LEISTUNG'], axis=1)
# data = data.merge(df3, left_index=True, right_index=True)

In [39]:
data = df

In [40]:
#%% Bag of Words
unique = data.ID.value_counts(normalize=False).index.tolist()
row_list = []
row_list1 = []
row_list2 = []
row_list3 = []
row_list4 = []
row_list5 = []
for i in unique:
    unique1 = data[data['ID']==i]
    df = unique1[['NUMMER', 'NUMMER_KAT', 'LEISTUNG']].astype(str)
    neu = df.apply(lambda x: x.str.split(expand=True).stack()).stack().value_counts()
    betr = unique1['RECHNUNGSBETRAG']
    #betr = betr[0:1]
    betr = betr.iloc[0]
    alt = unique1['ALTER']
    #alt = alt[1:2]
    alt = alt.iloc[0]
    gesch = unique1['GESCHLECHT']
    #gesch = gesch[2:3]
    gesch = gesch.iloc[0]
    fach = unique1['FACHRICHTUNG']
    #fach = fach[2:3]
    fach = fach.iloc[0]
    betrag = unique1['BETRAG']  
    #betrag = betrag[12:13]
    betrag = betrag.mean() ## !!!
    row_list1.append(betr)
    row_list2.append(alt)
    row_list3.append(gesch)
    row_list4.append(fach)
    row_list5.append(betrag)
    row_list.append(dict(neu))
df3 = pd.DataFrame(row_list)
df6 = pd.DataFrame(row_list1,columns=['RECHNUGSBETRAG'])
df7 = pd.DataFrame(row_list2,columns=['ALTER'])
df8 = pd.DataFrame(row_list3,columns=['GESCHLECHT'])
df9 = pd.DataFrame(row_list5,columns=['BETRAG'])
df10 = pd.DataFrame(row_list4,columns=['FACHRICHTUNG'])
df4 = pd.DataFrame(unique, columns=['ID'])
df5 = df4.merge(df6,left_index=True,right_index=True)
df5 = df5.merge(df3,left_index=True,right_index=True)
df5 = df5.merge(df7,left_index=True,right_index=True)
df5 = df5.merge(df8,left_index=True,right_index=True)
df5 = df5.merge(df9,left_index=True,right_index=True)
df5 = df5.merge(df10,left_index=True,right_index=True)

In [41]:
#%%
#Aneinander hängen für Modell 2
unique = data.ID.value_counts(normalize=False).index.tolist()
leer = []
for i in unique:
    test1 = data[data['ID']==i]
    for i in range(172):
        i += 1
        bez = 'NUMMER_KAT_' + str(i)
        test1[bez] = np.nan
        bez1 = 'NUMMER_' + str(i)
        test1[bez1] = np.nan
        bez2 = 'LEISTUNG_' + str(i)
        test1[bez1] = np.nan
    index = test1.index.values.astype(int)
    for j in range(1,len(index)):
        test2 = test1['NUMMER_KAT'][index[j]]
        test1.iloc[0,14+j] = test2
        test3 = test1['NUMMER'][index[j]]
        test1.iloc[0,186+j] = test3
        test4 = test1['LEISTUNG'][index[j]]
        test1.iloc[0,358+j] = test3
        k = index[j]
        test1 = test1.drop(labels=k, axis = 0)
    frames = [leer,test1,test2,test3]
    leer = pd.concat(frames)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

IndexError: single positional indexer is out-of-bounds

In [2]:
df = pd.read_csv('TransformedData.csv')
df.fillna(0, inplace=True)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['ID', 'target'], axis=1), df.target, random_state=0)

scaler = StandardScaler()
train_x = scaler.fit_transform(X_train)
test_x = scaler.transform(X_test)
test_y = y_test
train_y = y_train

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:616: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:459: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [50]:
#%%
'''Spalte Betrug einfügen; 0-> kein Betrug, 1 -> Betrug'''
df.loc [ df.KORREKTUR > 0 , 'Betrug' ] = 1
df.loc [ df.KORREKTUR == 0 , 'Betrug' ] = 0
df = df.drop(['KORREKTUR'],axis=1)
df = df.drop(['ART'],axis=1)
print("{:.3f} % sind Betrugsfälle ".format(np.sum(df['Betrug']) / df.shape[0] * 100))
print(df.head())

#%%
'''Datensatz aufteilen, so dass in beiden der gleich Anteil Betrug vorkommt'''
split = 0.25
anteil_trainingssatz = int((1-split) *df.shape[0])
train_x = df.iloc[:anteil_trainingssatz, 0:-1].values
train_y = df.iloc[:anteil_trainingssatz, -1].values
test_x = df.iloc[anteil_trainingssatz:, 0:-1].values
test_y = df.iloc[anteil_trainingssatz:, -1].values

print(train_x)
print("Größe Trainingssatz {}, Anzahl Betrug: {}, Prozentualer Anteil Betrug {:.5f}".format(train_x.shape[0], np.sum(train_y), np.sum(train_y)/train_x.shape[0]))
print("Größe Testsatz {}, Anzahl Betrug: {}, Prozentualer Anteil Betrug {:.5f}".format(test_x.shape[0], np.sum(test_y), np.sum(test_y)/test_y.shape[0]))

#%%
'''Normalisierung'''
mean = []
std = []
for i in range(train_x.shape[1]):
    mean.append(train_x[:,i].mean())
    std.append(train_x[:,i].std())
    train_x[:, i] = (train_x[:, i] - mean[-1]) / std[-1]
    test_x[:, i] =  (test_x[:, i] - mean[-1]) / std[-1]

2.000 % sind Betrugsfälle 
     ID  RECHNUNGSBETRAG      ALTER  GESCHLECHT  VERSICHERUNG  FACHRICHTUNG  \
0  ID_1       330.970001  53.570385           0             1             1   
1  ID_2       455.200012  83.382721           1             1             1   
2  ID_3       199.529999  69.567513           1             1             1   
3  ID_4       142.850006  69.556328           1             1             1   
4  ID_5       168.869995  18.873434           0             1             1   

   NUMMER NUMMER_KAT  TYP  ANZAHL  FAKTOR     BETRAG LEISTUNG  Betrug  
0   A_178      AA_10  0.0       2    2.30  24.400000      C_1     0.0  
1   A_765      AA_13  0.0       1    1.15   2.010000      C_6     0.0  
2  A_1978       AA_2  0.0       1    2.30  21.450001      C_1     0.0  
3  A_1257       AA_3  0.0       3    2.30  32.160000      C_1     0.0  
4   A_737      AA_13  0.0       1    1.15   3.350000      C_6     0.0  
[['ID_1' 330.9700012207031 53.57038497924805 0 1 1 'A_178' 'AA_10'

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:

#%%
ausgabe_zaehler = 1

'''Hyperparameter'''
lernrate = 0.001
epochen = 1000
batch = 1024
beta = 0.1
momentum = 0.2
#%%
'''Knoten in versteckter Schicht'''
n_hidden_1 = 15 #Knoten in erster Schicht
n_hidden_2 = 10 #Knoten in zweiter Schicht
n_hidden_3 = 5  #Knoten in dritter Schicht

'''Inputschicht'''
n_input = train_x.shape[1]

#%%
'''Platzhalter für Tensor'''
X = tf.placeholder("float", [None, n_input])

'''Gewichte - zufällig Standardnormal'''
gewicht = {
        'encoder_h1': tf.Variable(tf.random_normal([n_input , n_hidden_1])),
        'decoder_h1': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
        'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1 , n_hidden_2])),
        'decoder_h2': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
        'encoder_h3': tf.Variable(tf.random_normal([n_hidden_2 , n_hidden_3])),
        'decoder_h3': tf.Variable(tf.random_normal([n_hidden_3 , n_hidden_2]))
        }

'''Bias - zufällig Standardnormal'''
bias = {
        'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'decoder_b1': tf.Variable(tf.random_normal([n_input])),
        'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'decoder_b2': tf.Variable(tf.random_normal([n_hidden_1])),
        'encoder_b3': tf.Variable(tf.random_normal([n_hidden_3])),
        'decoder_b3': tf.Variable(tf.random_normal([n_hidden_2]))
        }

#%%
'''Encoder'''
def encoder(x):
    # erste Schicht
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, gewicht['encoder_h1']), bias['encoder_b1']))
    # dropout mit Rate pi
    layer_1 = tf.layers.dropout(layer_1,rate=0.2)
    # zweite Schicht
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, gewicht['encoder_h2']), bias['encoder_b2']))
    # dropout mit Rate pi
    layer_2 = tf.layers.dropout(layer_2,rate=0.2)
    # dritte Schicht
    layer_3 = tf.nn.tanh(tf.add(tf.matmul(layer_2, gewicht['encoder_h3']), bias['encoder_b3']))
    # dropout mit Rate pi
    layer_3 = tf.layers.dropout(layer_3,rate=0.2)
    return layer_3


#%%
'''Decoder'''
def decoder(x):
    # erste Schicht
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, gewicht['decoder_h3']), bias['decoder_b3']))
    # dropout mit Rate pi
    layer_1 = tf.layers.dropout(layer_1,rate=0.2)
    # zweite Schicht
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, gewicht['decoder_h2']), bias['decoder_b2']))
    # dropout mit Rate pi
    layer_2 = tf.layers.dropout(layer_2,rate=0.2)
    # dritte Schicht
    layer_3 = tf.nn.tanh(tf.add(tf.matmul(layer_2, gewicht['decoder_h1']), bias['decoder_b1']))
    # dropout mit Rate pi
    layer_3 = tf.layers.dropout(layer_3,rate=0.2)
    return layer_3

#%%
'''Autoencoder'''
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

#%%
'''Output Autoencoder'''
y_pred = decoder_op

#%%
'''gewünschter Output = Input'''
y_true = X

#%%
'''mittlerer quadratischer Fehler im batch'''
batch_mse = tf.reduce_mean(tf.pow(y_true - y_pred, 2), 1)

#%%
'''Fehlerfunktion'''
fehler = tf.reduce_mean(tf.pow(y_true - y_pred, 2))

'''Opitimierer'''
#Standardotimierer
optimierer = tf.train.RMSPropOptimizer(lernrate).minimize(fehler)

#Momentumoptimierer
#optimizer = tf.train.MomentumOptimizer(lernrate,momentum).minimize(fehler)

'''Regularisierung'''
#L2 Regularisierungsterm
reg = tf.nn.l2_loss(gewicht['encoder_h1']) + tf.nn.l2_loss(gewicht['encoder_h2']) + tf.nn.l2_loss(gewicht['decoder_h1']) + tf.nn.l2_loss(gewicht['decoder_h2'])
#reg = tf.nn.l2_loss(gewicht['encoder_h1']) + tf.nn.l2_loss(gewicht['encoder_h2']) + tf.nn.l2_loss(gewicht['decoder_h1']) + tf.nn.l2_loss(gewicht['decoder_h2']) + tf.nn.l2_loss(gewicht['decoder_h3']) + tf.nn.l2_loss(gewicht['decoder_h3'])

#Fehlerfunktion mit L2 Regularisierung
fehler = tf.reduce_mean(fehler + beta * reg)
#%%
'''Modell speichern'''
pfad = '.'
save_model = os.path.join(pfad, 'temp_modell.ckpt')
saver = tf.train.Saver()

#%%
'''Variablen initialisieren'''
init = tf.global_variables_initializer()

'''Session starten'''
with tf.Session() as sess:
    now = datetime.now()
    sess.run(init)
    total_batch = int(train_x.shape[0]/batch)
    #Trainingsepochen
    for epoche in range(epochen):
        #Schleife über alle batches
        for i in range(total_batch):
            #batch ziehen
            batch_idx = np.random.choice(train_x.shape[0], batch)
            batch_xs = train_x[batch_idx]
            #optimieren
            _, Fehler = sess.run([optimierer, fehler], feed_dict={X: batch_xs})
        # Display logs per epoch step
        if epoche % ausgabe_zaehler == 0:
            train_batch_mse = sess.run(batch_mse, feed_dict={X: train_x})
            print("Epoche:", '%04d' % (epoche+1),
                  "Fehler =", "{:.9f}".format(Fehler),
                  "Trainings-AUC =", "{:.6f}".format(auc(train_y, train_batch_mse)),
                  "Zeit =", "{}".format(datetime.now() - now))
    save_path = saver.save(sess, save_model)
    print("Fertig %s" % save_path)

#%%
'''Modell auf Testdatensatz testen'''
save_model = os.path.join(pfad, 'temp_modell.ckpt')
saver = tf.train.Saver()

with tf.Session() as sess:
    now = datetime.now()
    saver.restore(sess, save_model)
    test_batch_mse = sess.run(batch_mse, feed_dict={X: test_x})
    print("Test-AUC: {:.6f}".format(auc(test_y, test_batch_mse)))